In [33]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Circle
import os
import shutil
from PIL import Image

In [34]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Circle
import os
import shutil
from PIL import Image

def generate_random_dot_motion(num_dots, coherence, dot_size, dot_speed, direction, duration, frame_rate, screen_size, circle_radius, output_filename):
    # Convert direction to radians
    direction_rad = np.deg2rad(direction)

    # Initialize dot positions
    theta = 2 * np.pi * np.random.rand(num_dots)
    r = circle_radius * np.sqrt(np.random.rand(num_dots))
    dot_positions = np.column_stack((r * np.cos(theta), r * np.sin(theta)))

    # Initialize dot motion
    dx = dot_speed * np.cos(direction_rad)
    dy = -dot_speed * np.sin(direction_rad)

    # Create figure and axes
    fig, ax = plt.subplots()
    ax.set_xlim(-circle_radius, circle_radius)
    ax.set_ylim(-circle_radius, circle_radius)
    ax.set_aspect('equal')
    ax.axis('off')
    fig.patch.set_facecolor('black')
    ax.set_facecolor('black')

    # Plot initial dots
    dots = ax.scatter(dot_positions[:, 0], dot_positions[:, 1], s=dot_size, color='white')
    circle = Circle((0, 0), circle_radius, edgecolor='white', facecolor='none', linewidth=0.5)
    ax.add_patch(circle)

    # Number of frames
    num_frames = int(duration * frame_rate)

    def update(frame):
        nonlocal dot_positions

        # Determine which dots are coherent
        coherent_dots = np.random.rand(num_dots) < coherence

        # Update positions for coherent dots
        dot_positions[coherent_dots, 0] += dx
        dot_positions[coherent_dots, 1] += dy

        # Update positions for incoherent dots
        dot_positions[~coherent_dots] += (np.random.rand(np.sum(~coherent_dots), 2) - 0.5) * 2 * dot_speed

        # Wrap around dots that move out of the circular region
        #distances = np.sqrt(dot_positions[:, 0]**2 + dot_positions[:, 1]**2)
        #out_of_bounds = distances > circle_radius
        #theta_out = 2 * np.pi * np.random.rand(np.sum(out_of_bounds))
        #r_out = circle_radius * np.sqrt(np.random.rand(np.sum(out_of_bounds)))
        #dot_positions[out_of_bounds] = np.column_stack((r_out * np.cos(theta_out), r_out * np.sin(theta_out)))
        # Wrap around dots that move out of the circular region
        distances = np.sqrt(dot_positions[:, 0]**2 + dot_positions[:, 1]**2)
        out_of_bounds = distances > circle_radius
        # Calculate new positions for out-of-bounds dots
        dot_positions[out_of_bounds, 0] *= -1  # Reflect x-coordinate
        dot_positions[out_of_bounds, 1] *= -1  # Reflect y-coordinate
        # Check if reflected dots are still out of bounds and remove them
        distances = np.sqrt(dot_positions[:, 0]**2 + dot_positions[:, 1]**2)
        out_of_bounds = distances > circle_radius
        
        colors = np.array(['white'] * num_dots)
        colors[out_of_bounds] = 'black'

        # Update dot positions and colors in plot
        dots.set_offsets(dot_positions)
        dots.set_color(colors)


        return dots,

    # Create animation
    ani = animation.FuncAnimation(fig, update, frames=num_frames, blit=True)

    # Temporary directory to save frames
    temp_dir = "temp_frames"
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
    os.makedirs(temp_dir)

    # Save each frame as an image
    for i in range(num_frames):
        update(i)
        plt.savefig(f"{temp_dir}/frame_{i:04d}.png", dpi=100)

    plt.close()

    # Convert images to MP4 using ffmpeg
    #os.system(f"ffmpeg -r {frame_rate} -i {temp_dir}/frame_%04d.png -vcodec libx264 -y random_dot_motion_circle.mp4")
    os.system(f"ffmpeg -r {frame_rate} -i {temp_dir}/frame_%04d.png -vcodec libx264 -y {output_filename}")


    # Clean up temporary frames
    shutil.rmtree(temp_dir)

# Example u

In [59]:
n_dots = 1000
dot_coh = .9
dot_size = 10
dot_angles = [180,270,0,90]
dot_speeds = [1,2,3,4]
dot_sets = [1]
for set in range(np.size(dot_sets)):
    for angle in range(np.size(dot_angles)):
        for speed in range(np.size(dot_speeds)):
            dot_speed = 2*dot_speeds[speed]
            dot_angle = dot_angles[angle]
            f_name = 'D' + str(angle+1) + '_S' + str(speed+1) + '_' + str(set+1) + '.mp4'
            generate_random_dot_motion(n_dots, dot_coh, dot_size, dot_speed, dot_angle, 5, 60, [800, 600], 200, f_name)

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with clang version 12.0.0
  configuration: --prefix=/Users/ktietz/demo/mc3/conda-bld/ffmpeg_1628925491858/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=arm64-apple-darwin20.0.0-clang --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57

In [58]:
n_dots = 1000
dot_coh = .9
dot_size = 10
dot_speed = 1
dot_angle = 180

f_name = 'D' + str(angle+1) + '_S' + str(speed+1) + '_' + str(set+1) + '.mp4'

generate_random_dot_motion(n_dots, dot_coh, dot_size, dot_speed, dot_angle, 5, 60, [800, 600], 200, f_name)

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with clang version 12.0.0
  configuration: --prefix=/Users/ktietz/demo/mc3/conda-bld/ffmpeg_1628925491858/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=arm64-apple-darwin20.0.0-clang --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57